In [2]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x58f04000 @  0x7f2614de42a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [0]:
import torch

In [4]:
cuda = torch.cuda.is_available()
print(cuda)

True


In [0]:
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.datasets as dsets
from torch.autograd import Variable
import numpy as np
import random as rand
%matplotlib inline
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [7]:
torch.manual_seed(10)
if cuda:
  torch.cuda.manual_seed(10)
'''
LOADING DATASET

'''

trainset = dsets.CIFAR10('./data', train= True, download= True,transform = transforms.ToTensor())
testset = dsets.CIFAR10('./data', train= False, download = True, transform = transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [8]:
len(trainset), len(testset), trainset[0][0].size(), trainset[0][1]

(50000, 10000, torch.Size([3, 32, 32]), 6)

In [9]:
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
CLASSES[trainset[0][1]]

'frog'

In [0]:
batch_size = 50
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, shuffle =True, num_workers =10)
test_loader=torch.utils.data.DataLoader(dataset=testset, batch_size=batch_size,shuffle=True,num_workers=10)


In [0]:
'''
CREATE MODEL

'''

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.input_hidden1 = nn.Linear(3*32*32,530)
    self.input_hidden1_dropout = nn.Dropout(0.2)
    self.hidden1_hidden2 = nn.Linear(530,300)
    self.hidden1_hidden2_dropout = nn.Dropout(0.2)
    self.hidden2_output = nn.Linear(300,10)
    
  def forward(self, x, activ):
    x= x.view(-1,3*32*32) #reshaping the input
    x= activ(self.input_hidden1(x))
    x= self.input_hidden1_dropout(x)
    x= activ(self.hidden1_hidden2(x))
    x= self.hidden1_hidden2_dropout(x)
    return self.hidden2_output(x)
    

In [0]:
model = Net()
if cuda:
  model.cuda()

In [0]:
class CIFAR_MODEL:
  def __init__(self, model, learning_rate, optimizer_value, activ, loss_type):
    self.model = model
    self.lr = learning_rate
    
    weightdecay = 0
    if loss_type == 'l2': #L2 regularization on loss by weightdecay
      weightdecay=0.01
      
    if(optimizer_value == 'sgd'):
      self.optimizer = optim.SGD(self.model.parameters(), lr= learning_rate, momentum = 0.9, weight_decay=weightdecay)
    elif(optimizer_value == 'adam'):
      self.optimizer = optim.Adam(self.model.parameters(), lr= learning_rate, weight_decay = weightdecay)
    elif(optimizer_value == 'adagrad'):
      self.optimizer = optim.Adagrad(self.model.parameters(), lr= learning_rate, weight_decay = weightdecay )
    elif(optimizer_value == 'adadelta'):
      self.optimizer = optim.Adadelta(self.model.parameters(), lr= learning_rate, weight_decay = weightdecay)
    else:
      self.optimizer = optim.RMSprop(self.model.parameters(), lr= learning_rate, weight_decay = weightdecay)
    
  def train(self, epoch, loss_type, log_interval=100):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
      if cuda:
        data,target = data.cuda(), target.cuda()
      data, target = Variable(data), Variable(target)
      self.optimizer.zero_grad()
      output = model(data,activ)
      loss = criterion(output, target)
      if loss_type == 'l1': #L1 regularisation on loss
        nn.L1Loss(loss)
      loss.backward()
      self.optimizer.step()
      if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
    
  def test(self, loss_vector, accuracy_vector):
    model.eval()
    test_loss, correct = 0,0
    for (data,target) in test_loader:
      if cuda:
        data, target = data.cuda(), target.cuda()
      data, target = Variable(data, volatile = True), Variable(target)
      output = model(data, activ)
      prediction = output.data.max(1)[1]
      confusionmatrix = confusion_matrix(target,prediction)
      test_loss+= criterion(output,target).data[0]
      pred = output.data.max(1)[1]
      correct+= pred.eq(target.data).cpu().sum()
    
    test_loss/=len(test_loader)
    loss_vector.append(test_loss)
    accuracy = 100.*correct/len(test_loader.dataset)
    accuracy_vector.append(accuracy)
    
    print('Confusion_Matrix', confusionmatrix)
    print('Accuracy Score :',accuracy_score(target, prediction))
    print('\nTest set: Average loss: {:.4f}, Accuracy: ({:.0f}%)\n'.format(
        test_loss, accuracy))

In [0]:
'''
INSTANTIATE LOSS CLASS
  
'''

criterion = nn.CrossEntropyLoss()
if cuda:
  criterion = criterion.cuda()
  
def softmax(x):
  x = x - np.max(x, axis=1).reshape((-1,1))
  x = np.exp(x)
  return x / np.sum(x, axis=1).reshape((-1, 1))
  
def hingeloss(x,y):
  '''
  We were trying to impleent it like this but we were facing some error
  return (torch.clamp(torch.mm(y,x)-1,min = 0))
  '''
  return(max(0, 1-y*x))
  
def softmax_hingeloss(x,y):
  return softmax(x) + hingeloss(x,y)



In [44]:
%%time
epochs = 1
lossv,accuv = [],[]
learning_rate= 0.001
loss_type = 'l2'
optimizer_list = ['sgd', 'adam', 'adagrad', 'adadelta', 'rmsprop']
activation = [F.relu,F.tanh,F.sigmoid]

for optimiz in optimizer_list:
  for activ in activation:
    print('optimizer:',optimiz,'activation function:',activ)
    cifar = CIFAR_MODEL(model, learning_rate, optimiz, activ, loss_type)
    cifar.train(epoch,loss_type)
    cifar.test(lossv,accuv)

optimizer: sgd activation function: <function relu at 0x7f10ae885048>


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.025589
Train Epoch: 1 [5000/50000 (10%)]	Loss: 1.986771
Train Epoch: 1 [10000/50000 (20%)]	Loss: 1.937564
Train Epoch: 1 [15000/50000 (30%)]	Loss: 1.738441
Train Epoch: 1 [20000/50000 (40%)]	Loss: 1.939936
Train Epoch: 1 [25000/50000 (50%)]	Loss: 1.955646
Train Epoch: 1 [30000/50000 (60%)]	Loss: 1.761591
Train Epoch: 1 [35000/50000 (70%)]	Loss: 1.770161
Train Epoch: 1 [40000/50000 (80%)]	Loss: 1.764068
Train Epoch: 1 [45000/50000 (90%)]	Loss: 1.808837


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Confusion_Matrix [[3 0 0 0 0 0 0 1 0 0]
 [0 3 0 0 0 0 0 0 1 2]
 [1 0 0 0 3 1 2 1 1 0]
 [0 0 0 0 0 2 0 0 0 1]
 [0 0 0 0 1 0 1 1 1 0]
 [0 0 0 0 1 2 0 0 1 0]
 [0 0 1 0 0 0 2 1 0 0]
 [1 0 1 1 1 1 0 1 0 1]
 [1 0 0 0 0 1 0 0 1 2]
 [0 0 0 0 0 0 0 0 0 4]]
Accuracy Score : 0.34

Test set: Average loss: 1.8589, Accuracy: (33%)

optimizer: sgd activation function: <function tanh at 0x7f10ae8859d8>


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Train Epoch: 1 [0/50000 (0%)]	Loss: 1.886170
Train Epoch: 1 [5000/50000 (10%)]	Loss: 1.827944
Train Epoch: 1 [10000/50000 (20%)]	Loss: 1.645394
Train Epoch: 1 [15000/50000 (30%)]	Loss: 1.897365
Train Epoch: 1 [20000/50000 (40%)]	Loss: 1.964714
Train Epoch: 1 [25000/50000 (50%)]	Loss: 1.786389
Train Epoch: 1 [30000/50000 (60%)]	Loss: 1.963215
Train Epoch: 1 [35000/50000 (70%)]	Loss: 1.894994
Train Epoch: 1 [40000/50000 (80%)]	Loss: 1.990583
Train Epoch: 1 [45000/50000 (90%)]	Loss: 1.750635
Confusion_Matrix [[2 0 0 0 0 0 1 0 1 0]
 [0 3 0 0 0 0 0 0 1 2]
 [1 1 0 1 3 1 0 0 0 0]
 [0 0 1 0 0 1 3 0 0 0]
 [0 0 1 0 3 0 0 0 0 0]
 [0 0 0 0 3 2 0 0 0 0]
 [0 0 0 0 0 1 3 1 0 1]
 [0 0 2 0 0 0 0 2 0 1]
 [0 0 0 0 0 0 0 0 1 1]
 [0 1 0 0 1 0 1 0 1 2]]
Accuracy Score : 0.36

Test set: Average loss: 1.8429, Accuracy: (34%)

optimizer: sgd activation function: <function sigmoid at 0x7f10ae885a60>


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.338853
Train Epoch: 1 [5000/50000 (10%)]	Loss: 2.280159
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.271400
Train Epoch: 1 [15000/50000 (30%)]	Loss: 2.197170
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.193701
Train Epoch: 1 [25000/50000 (50%)]	Loss: 2.197694
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.238880
Train Epoch: 1 [35000/50000 (70%)]	Loss: 2.267112
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.159996
Train Epoch: 1 [45000/50000 (90%)]	Loss: 2.233701
Confusion_Matrix [[2 0 0 2 0 0 0 0 1 0]
 [0 1 0 1 0 1 1 0 0 0]
 [2 4 0 1 0 0 0 0 1 0]
 [0 1 0 3 0 1 0 0 0 0]
 [0 0 0 3 0 2 0 0 0 0]
 [0 0 0 4 0 2 0 0 0 0]
 [0 0 0 4 0 1 0 0 0 0]
 [1 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 1 0 0 3 2]
 [0 1 0 0 0 0 0 0 0 2]]
Accuracy Score : 0.26

Test set: Average loss: 2.1779, Accuracy: (25%)

optimizer: adam activation function: <function relu at 0x7f10ae885048>
Train Epoch: 1 [0/50000 (0%)]	Loss: 1.998143
Train Epoch: 1 [5000/50000 (10%)]	Loss: 2.044054
Train Epoch: 1 [10000

In [0]:
torch.save(model.state_dict(), "model_assignment3.th")

In [46]:
the_model = Net()
the_model.load_state_dict(torch.load("model_assignment3.th"))
print(the_model)


Net(
  (input_hidden1): Linear(in_features=3072, out_features=530, bias=True)
  (input_hidden1_dropout): Dropout(p=0.2)
  (hidden1_hidden2): Linear(in_features=530, out_features=300, bias=True)
  (hidden1_hidden2_dropout): Dropout(p=0.2)
  (hidden2_output): Linear(in_features=300, out_features=10, bias=True)
)


In [47]:
torch.save(model, "model_assignment3.th.2")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [48]:
the_model2 = torch.load("model_assignment3.th.2")
print(the_model2)

Net(
  (input_hidden1): Linear(in_features=3072, out_features=530, bias=True)
  (input_hidden1_dropout): Dropout(p=0.2)
  (hidden1_hidden2): Linear(in_features=530, out_features=300, bias=True)
  (hidden1_hidden2_dropout): Dropout(p=0.2)
  (hidden2_output): Linear(in_features=300, out_features=10, bias=True)
)
